# Metabolites

## Description
In this notebook we create a dataframe containing all the available information for the metabolites accounted in our reconstruction.|
1. [Generation of Metabolites dataset](#Generation)

In [2]:
# Import libraries
import gspread
import pandas as pd
import numpy as np
import cobra
from cobra.io import read_sbml_model

from tqdm.notebook import tqdm

/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [44]:
# Define functions

def df_to_dict(df, key_col):
    """
    This function takes a pandas dataframe and a key column, and returns a dictionary
    with the key column as the dictionary keys and the rest of the columns as the values.
    """
    # Create an empty dictionary to hold the key-value pairs
    my_dict = {}
    
    # Loop through each row in the dataframe
    for index, row in df.iterrows():
        # Use the value in the key column as the dictionary key
        key_value = row[key_col]
        
        # Use the rest of the columns as the dictionary values
        value_dict = row.drop(key_col).to_dict()
        
        # Add the key-value pair to the dictionary
        my_dict[key_value] = value_dict
    
    return my_dict

## A. Generation of Metabolites dataset
We start by creating a list of all the metabolites included in the reactions of our reconstruction (1). Then we create a dataset containing all the metabolites info from Recon3D, iCHO2291 and iCHO1766 models, including supplementary information from Recon 3D (2). Now we can map back this information into the metabolites from our reconstruction and generate an excell file for uploading into Google Sheets (3).

### 1. Retrieve a list of all the metabolites from our reconstruction
The list of all the reactions and the metabolites involved are in the Rxns Sheet in the Google Sheet.

In [66]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
rxns_sheet = cho_recon.worksheet('Rxns')

In [67]:
# We can extract the data using the get_all_records method and create a pd DataFrame
df = pd.DataFrame(rxns_sheet.get_all_records())
df

,Curated,Reaction,Reaction Name,Reaction Formula,Subsystem,GPR_hef,GPR_fou,GPR_yeo,GPR_Recon3D,GPR_final,Conf. Score,Curation Notes,References
0,PD,10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal",10fthf5glu_c --> 10fthf5glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,
1,PD,10FTHF5GLUtm,"5-glutamyl-10FTHF transport, mitochondrial",10fthf5glu_m --> 10fthf5glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,
2,PD,10FTHF6GLUtl,"6-glutamyl-10FTHF transport, lysosomal",10fthf6glu_c --> 10fthf6glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,
3,PD,10FTHF6GLUtm,"6-glutamyl-10FTHF transport, mitochondrial",10fthf6glu_m --> 10fthf6glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,
4,PD,10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal",10fthf7glu_c --> 10fthf7glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,,r2537,Utilized transport,lnlncgcoa_c <=> lnlncgcoa_r,Transport,,,,,,,,
8185,,r2538,Utilized transport,dlnlcgcoa_c <=> dlnlcgcoa_r,Transport,,,,,,,,
8186,,r2539,Postulated transport reaction,L2aadp6sa_c + L2aadp_m <=> L2aadp6sa_m + L2aadp_c,Transport,,,,,,,,
8187,PD,ALLTTtm,"Allantoate transport via diffusion, mitochondria",alltt_c <=> alltt_m,"Transport, mitochondria",,,,,,1,The transport of Allantoate from the cytoplasm...,


In [68]:
# Create a cobra model to identify the metabolites involved in our reconstruction
model = cobra.Model("iCHOxxxx")
lr = []

for _, row in df.iterrows():
    r = cobra.Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

Name,iCHOxxxx
Memory address,2859eded0
Number of metabolites,0
Number of reactions,8189
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [70]:
# With the built in function "build_reaction_from_string" we can identify the metabolites
for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(df['Reaction Formula'][i])

  0%|          | 0/8189 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [71]:
# We first create a list of the metabolites and then a pandas df with it
metabolites_list = []
for met in model.metabolites:
    metabolites_list.append(met.id)
    
metabolites = pd.DataFrame(metabolites_list, columns =['BiGG ID'])
metabolites

,BiGG ID
0,10fthf5glu_c
1,10fthf5glu_l
2,10fthf5glu_m
3,10fthf6glu_c
4,10fthf6glu_l
...,...
5426,HC02214_c
5427,HC02216_c
5428,HC02217_c
5429,fdxrd_c


### 2. Retrieve information from all the metabolites on Recon3D, iCHO2291 and iCHO1766
We use two datasets for this, first we take information from the Recon3D.xml, iCHO2291.xml and iCHO1766 files from which we get the metabolite ID, Name, Formula and Compartment. We then add the metadata for the available metabolites from Recon3D supplementary files.

In [72]:
# read the Recon3D model
recon3d_model = read_sbml_model('../Data/GPR_Curation/Recon3D.xml')

In [73]:
# Generate a dataset containing all the metabolites, chemical formula of each metabolite and compartment
num_rows = len(recon3d_model.metabolites)
recon3d_model_metabolites = pd.DataFrame(index=range(num_rows), columns=['BiGG ID', 'Name', 'Formula', 'Compartment'])
for i,met in enumerate(recon3d_model.metabolites):
    id_ = met.id
    name = met.name
    formula = met.formula
    comp = met.compartment
    recon3d_model_metabolites.iloc[i] = [id_, name, formula, comp]

In [74]:
recon3d_model_metabolites

,BiGG ID,Name,Formula,Compartment
0,10fthf_c,10-Formyltetrahydrofolate,C20H21N7O7,c
1,10fthf_l,10-Formyltetrahydrofolate,C20H21N7O7,l
2,10fthf_m,10-Formyltetrahydrofolate,C20H21N7O7,m
3,11docrtsl_c,11docrtsl c,C21H30O4,c
4,11docrtsl_m,11docrtsl c,C21H30O4,m
...,...,...,...,...
5830,caproic_e,Caproic Acid,C6H11O2,e
5831,1a25dhvitd2_c,"1-alpha,25-Dihydroxyvitamin D2",C28H44O3,c
5832,1a25dhvitd2_e,"1-alpha,25-Dihydroxyvitamin D2",C28H44O3,e
5833,protein_c,Torasemide-M3,None,c


In [75]:
# read the Yeo's model
iCHO2291_model = read_sbml_model('../Data/Reconciliation/models/iCHO2291.xml')

In [76]:
# Generate a dataset containing all the metabolites, chemical formula of each metabolite and compartment from Yeo's model
num_rows = len(iCHO2291_model.metabolites)
iCHO2291_model_metabolites = pd.DataFrame(index=range(num_rows), columns=['BiGG ID', 'Name', 'Formula', 'Compartment'])
for i,met in enumerate(iCHO2291_model.metabolites):
    id_ = met.id
    name = met.name
    formula = met.formula
    comp = met.compartment
    iCHO2291_model_metabolites.iloc[i] = [id_, name, formula, comp]
    
iCHO2291_model_metabolites['BiGG ID'] = iCHO2291_model_metabolites['BiGG ID'].str.replace("[", "_", regex=False)
iCHO2291_model_metabolites['BiGG ID'] = iCHO2291_model_metabolites['BiGG ID'].str.replace("]", "", regex=False)
iCHO2291_model_metabolites

,BiGG ID,Name,Formula,Compartment
0,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c
1,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l
2,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m
3,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c
4,10fthf6glu_l,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,l
...,...,...,...,...
3967,Rtotal3crn_c,Rtotal3crn[c],CO2R3C7H14NO2,c
3968,Rtotal3crn_m,Rtotal3crn[m],CO2R3C7H14NO2,m
3969,Rtotalcrn_c,Rtotalcrn[c],CO2RC7H14NO2,c
3970,Rtotalcrn_m,Rtotalcrn[m],CO2RC7H14NO2,m


In [77]:
# read Hefzi's model
iCHO1766_model = read_sbml_model('../Data/Reconciliation/models/iCHOv1_final.xml')

In [78]:
# Generate a dataset containing all the metabolites, chemical formula of each metabolite and compartment from Hefzi's model
num_rows = len(iCHO1766_model.metabolites)
iCHO1766_model_metabolites = pd.DataFrame(index=range(num_rows), columns=['BiGG ID', 'Name', 'Formula', 'Compartment'])
for i,met in enumerate(iCHO1766_model.metabolites):
    id_ = met.id
    name = met.name
    formula = met.formula
    comp = met.compartment
    iCHO1766_model_metabolites.iloc[i] = [id_, name, formula, comp]

iCHO1766_model_metabolites

,BiGG ID,Name,Formula,Compartment
0,coke_r,cocaine,C17H21NO4,r
1,h2o_r,H2O,H2O,r
2,bz_r,benzoate,C7H5O2,r
3,egme_r,ecgonine methyl ester,C10H17NO3,r
4,h_r,proton,H,r
...,...,...,...,...
4451,igg_g,igg[g],None,g
4452,nicrns_c,Nicotinate D-ribonucleoside,C11H13NO6,c
4453,bilglcur_r,Bilirubin monoglucuronide,C39H44N4O12,r
4454,pcollglys_c,Procollagen L-lysine,C7H14N3O2R2,c


In [79]:
models_metabolites = pd.concat([recon3d_model_metabolites, iCHO2291_model_metabolites, iCHO1766_model_metabolites])
models_metabolites = models_metabolites.groupby('BiGG ID').first()
models_metabolites = models_metabolites.reset_index(drop = False)
models_metabolites

,BiGG ID,Name,Formula,Compartment
0,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c
1,10fthf5glu_e,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,e
2,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l
3,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m
4,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c
...,...,...,...,...
8527,zym_int2_r,Zymosterol intermediate 2 C27H42O,C27H42O,r
8528,zymst_c,Zymosterol C27H44O,C27H44O,c
8529,zymst_r,Zymosterol C27H44O,C27H44O,r
8530,zymstnl_c,5alpha-cholest-8-en-3beta-ol,C27H46O,c


In [54]:
#Generation of a dataset containing all the information from Recon3D metabolites Supplementary Data.
recon3d_metabolites_meta = pd.read_excel('../Data/Metabolites/metabolites.recon3d.xlsx', header = 0)
recon3d_metabolites_meta['BiGG ID'] = recon3d_metabolites_meta['BiGG ID'].str.replace("[", "_", regex=False)
recon3d_metabolites_meta['BiGG ID'] = recon3d_metabolites_meta['BiGG ID'].str.replace("]", "", regex=False)
recon3d_metabolites_meta

,BiGG ID,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
0,10fthf5glu_c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10fthf5glu_l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10fthf5glu_m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10fthf6glu_c,NaN,NaN,NaN,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10fthf6glu_l,NaN,NaN,NaN,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,NaN,NaN,CID100015232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5283,NaN,NaN,CID100123634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5284,NaN,NaN,CID100206527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5285,NaN,NaN,CID105479141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Transformation of the "recon3d_metabolites_meta" into a dict to map it into the "recon3d_model_metabolites"
recon3dmet_dict = df_to_dict(recon3d_metabolites_meta, 'BiGG ID')

In [56]:
# Mapping into the "recon3d_model_metabolites" dataset
models_metabolites[['KEGG','CHEBI', 'PubChem','Inchi', 'Hepatonet', 'EHMNID', 'SMILES', 'INCHI2',
                          'CC_ID','Stereoisomer Information of Metabolite Identified', 'Charge of the Metabolite Identified',
    'CID_ID','PDB (ligand-expo) Experimental Coordinates  File Url', 'Pub Chem Url',
    'ChEBI Url']] = models_metabolites['BiGG ID'].apply(lambda x: pd.Series(recon3dmet_dict.get(x, None), dtype=object))

In [57]:
models_metabolites

,BiGG ID,Name,Formula,Compartment,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
0,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10fthf5glu_e,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c,NaN,NaN,NaN,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8527,zym_int2_r,Zymosterol intermediate 2 C27H42O,C27H42O,r,C05437,18252.0,22298942.0,InChI=1S/C27H42O/c1-18(2)7-6-8-19(3)23-11-12-2...,NaN,NaN,[H][C@@]12CCC3=C(CC[C@]4(C)[C@]([H])(CC[C@@]34...,InChI=1S/C27H42O/c1-18(2)7-6-8-19(3)23-11-12-2...,NaN,NaN,Neutral,22298942.0,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/2229...,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...
8528,zymst_c,Zymosterol C27H44O,C27H44O,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8529,zymst_r,Zymosterol C27H44O,C27H44O,r,C05437,18252.0,92746.0,InChI=1S/C27H44O/c1-18(2)7-6-8-19(3)23-11-12-2...,HC01451,C05437,[H][C@@]12CCC3=C(CC[C@]4(C)[C@]([H])(CC[C@@]34...,InChI=1S/C27H44O/c1-18(2)7-6-8-19(3)23-11-12-2...,NaN,NaN,Neutral,92746.0,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/92746,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...
8530,zymstnl_c,5alpha-cholest-8-en-3beta-ol,C27H46O,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Transform the final Recon3D Metabolites dataset into a dictionary to map it into our dataset
final_met_dict = df_to_dict(models_metabolites, 'BiGG ID')

### 3. Add all the metabolites information into our metabolites dataset
With the dictionary created in **Step 2** we can use the information to map it in the metabolites dataset created in **Step 1** which contains all the metabolites of our reconstruction.

In [59]:
metabolites[['Name', 'Formula', 'Compartment', 'KEGG','CHEBI', 'PubChem','Inchi', 'Hepatonet', 'EHMNID', 'SMILES',
             'INCHI2','CC_ID','Stereoisomer Information of Metabolite Identified', 'Charge of the Metabolite Identified',
    'CID_ID','PDB (ligand-expo) Experimental Coordinates  File Url', 'Pub Chem Url',
    'ChEBI Url']] = metabolites['BiGG ID'].apply(lambda x: pd.Series(final_met_dict.get(x, None), dtype=object))

In [60]:
# Update the Compartment column in the final dataset
for i,row in metabolites.iterrows():
    if row['Compartment'] == 'c':
        metabolites.loc[i, 'Compartment'] = 'c - cytosol'
    if row['Compartment'] == 'l':
        metabolites.loc[i, 'Compartment'] = 'l - lysosome'
    if row['Compartment'] == 'm':
        metabolites.loc[i, 'Compartment'] = 'm - mitochondria'
    if row['Compartment'] == 'r':
        metabolites.loc[i, 'Compartment'] = 'r - endoplasmic reticulum'
    if row['Compartment'] == 'e':
        metabolites.loc[i, 'Compartment'] = 'e - extracellular space'
    if row['Compartment'] == 'x':
        metabolites.loc[i, 'Compartment'] = 'x - peroxisome/glyoxysome'
    if row['Compartment'] == 'n':
        metabolites.loc[i, 'Compartment'] = 'n - nucleus'
    if row['Compartment'] == 'g':
        metabolites.loc[i, 'Compartment'] = 'g - golgi apparatus'
    if row['Compartment'] == 'im':
        metabolites.loc[i, 'Compartment'] = 'im - intermembrane space of mitochondria'

In [ ]:
metabolites.to_excel('../Data/Metabolites/metabolites.xlsx')

### 4. Unique metabolite identification

In [4]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction + Recon3D')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
metabolites_sheet = cho_recon.worksheet('Metabolites')

# We can extract the data using the get_all_records method and create a pd DataFrame
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())
metabolites

,Curated,BiGG ID,Name,Formula,Compartment,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
0,,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c - cytosol,,,,,,,,,,,,,,,
1,,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l - lysosome,,,,,,,,,,,,,,,
2,,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m - mitochondria,,,,,,,,,,,,,,,
3,,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c - cytosol,,,,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,,,,,,,,,,,
4,,10fthf6glu_l,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,l - lysosome,,,,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8661,,HC02098_c,3-Hydroxystearoyl-ACP,C18H35O2SR,c - cytosol,C16220,,,,,,CCCCCCCCCCCCCCCC(O)CC(=O)S[*],,,,Neutral,,,,https://www.ebi.ac.uk/chebi/searchId.do;jsessi...
8662,,HC02099_c,(2E)-Octadecenoyl-ACP,C18H33OSR,c - cytosol,C16221,,,,,,CCCCCCCCCCCCCCC\C=C\C(=O)S[*],,,,Neutral,,,,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...
8663,,HC01988_c,Stearoyl-ACP,X,c - cytosol,C04088,,,,,,,,,,,,,,
8664,,HC10856_m,"Trans,Cis-Octadeca-2,9-Dienoyl Coenzyme A",C39H62N7O17P3S,m - mitochondria,,,,,,,CCCCCCCC\C=C/CCCCC\C=C\C(=O)SCCNC(=O)CCNC(=O)[...,InChI=1S/C39H66N7O17P3S/c1-4-5-6-7-8-9-10-11-1...,,,Neutral,86289273,,https://pubchem.ncbi.nlm.nih.gov/compound/8628...,https://www.ebi.ac.uk/chebi/searchId.do;jsessi...


In [5]:
print("Duplicated rxns by BiGG ID = ", len(metabolites['BiGG ID']) - len(metabolites['BiGG ID'].unique()))
print("Duplicated rxns by Name = ", len(metabolites['Name']) - len(metabolites['Name'].unique()))
print("Duplicated rxns by Formula = ", len(metabolites['Formula']) - len(metabolites['Formula'].unique()))
print("Duplicated rxns by KEGG = ", len(metabolites['KEGG']) - len(metabolites['KEGG'].unique()))

Duplicated rxns by BiGG ID =  0
Duplicated rxns by Name =  3752
Duplicated rxns by Formula =  5837
Duplicated rxns by KEGG =  7536


In [6]:
# Group the metabolites by formula and create an excel file to curate them manually.
grouped_by_formula = metabolites.groupby('Formula', group_keys=True).apply(lambda x:x)
grouped_by_formula.to_excel('../Data/Metabolites/grouped_by_formula_recon3d.xlsx')

## B. Metabolites Curation
In this second part of the notebook we curate missing information in the metabolites dataset generated above. Since many metabolites have been manually curated in the "Metabolites" google sheet file, we generate a new dataframe using the gspread library to obtain the metabolites dataset with all the changes

### 5. Generate new dataframe from the "Metabolites" sheet 

In [6]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
metabolites_sheet = cho_recon.worksheet('Metabolites')

# We can extract the data using the get_all_records method and create a pd DataFrame
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())
metabolites

,Curated,BiGG ID,Name,Formula,Compartment,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
0,,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c - cytosol,,,,,,,,,,,,,,,
1,,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l - lysosome,,,,,,,,,,,,,,,
2,,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m - mitochondria,,,,,,,,,,,,,,,
3,,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c - cytosol,,,,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,,,,,,,,,,,
4,,10fthf6glu_l,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,l - lysosome,,,,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5426,,HC02216_c,Prostaglandin-f2beta,C20H33O5,c - cytosol,,28922,439702,InChI=1S/C20H34O5/c1-2-3-6-9-15(21)12-13-17-16...,,,CCCCC[C@H](O)\C=C\[C@H]1[C@H](O)C[C@@H](O)[C@@...,InChI=1S/C20H34O5/c1-2-3-6-9-15(21)12-13-17-16...,,,Neutral,5280506,,https://pubchem.ncbi.nlm.nih.gov/compound/5280506,https://www.ebi.ac.uk/chebi/searchId.do;jsessi...
5427,,HC01361_e,Dihydroneopterin,C9H13N5O4,e - extracellular space,C04874,17001,,InChI=1S/C9H13N5O4/c10-9-13-7-5(8(18)14-9)12-3...,,,Nc1nc2c(c(=O)[nH]1)N=C(C(O)C(O)CO)CN2,,,,,,,,
5428,,HC02217_c,Prostaglandin-g2,C20H31O6,c - cytosol,,27647,,InChI=1S/C20H32O6/c1-2-3-6-9-15(24-23)12-13-17...,,C05956,CCCCC[C@H](OO)\C=C\[C@H]1[C@H]2C[C@H](OO2)[C@@...,InChI=1S/C20H32O6/c1-2-3-6-9-15(24-23)12-13-17...,PGX,,Neutral,5280883,http://ligand-expo.rcsb.org/pyapps/ldHandler.p...,https://pubchem.ncbi.nlm.nih.gov/compound/5280883,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...
5429,PD,fdxo_2_2_c,Oxidized ferredoxin,Fe2S2X,c - cytosol,C00139,17908,,,,,[*][*][Fe+3]1([*][*])[S-2][Fe+3]([*][*])([*][*...,,,,,,,,


In [66]:
# Get BiGG descriptive names
import requests
from bs4 import BeautifulSoup
import time

# Unknown Mets: metabolites without names
unkown_mets = metabolites[metabolites['Name'] == '']

Descriptive_Names = [''] * len(unkown_mets)
Formulae = [''] * len(Descriptive_Names)
Changed = [True] * len(Descriptive_Names)

for Met_Counter, metID in enumerate(tqdm(unkown_mets['BiGG ID'].iloc[:])):
    print(Met_Counter)
    input_str = metID[:-2]
    response = requests.get(f"http://bigg.ucsd.edu/universal/metabolites/{input_str}")
    time.sleep(1)
    # Check if the request was successful
    if response.status_code != 200:
        D_Name = "BiGG ID not found in BiGG"
        Formulae_B = "BiGG ID not found in BiGG"
        Changed[Met_Counter] = False       
    else:    
        soup = BeautifulSoup(response.content, 'html.parser')
        N_Header = soup.find('h4', string='Descriptive name:')
        D_Name = N_Header.find_next_sibling('p').text
        N_Formulae = soup.find('h4', string='Formulae in BiGG models: ')
        Formulae_B = N_Formulae.find_next_sibling('p').text    
        if D_Name is None:
            D_Name = "Name not found in BiGG"            
        elif Formulae_B is None:
            Formulae_B = "Formula not found in BiGG"                
    Descriptive_Names[Met_Counter] = D_Name
    Formulae[Met_Counter] = Formulae_B

  0%|          | 0/160 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159


### Update empty metabolites

In [67]:
for Met_Counter, metID in enumerate(unkown_mets['BiGG ID']):
    print('before',unkown_mets['BiGG ID'].iloc[Met_Counter])
    print('before',unkown_mets['Formula'].iloc[Met_Counter])
    print('before',unkown_mets['Name'].iloc[Met_Counter])
    if unkown_mets['Formula'].iloc[Met_Counter] == '':
        unkown_mets['Formula'].iloc[Met_Counter] = Formulae[Met_Counter]  
    unkown_mets['Name'].iloc[Met_Counter] = Descriptive_Names[Met_Counter]
    print('..............................................')
    print('after',unkown_mets['BiGG ID'].iloc[Met_Counter])
    print('after',unkown_mets['Formula'].iloc[Met_Counter])
    print('after',unkown_mets['Name'].iloc[Met_Counter])
    print('..............................................')
    print('..............................................')
    print('..............................................')

before 3hoc101_7Zcoa_m
before C31H48N7O18P3S
before 
..............................................
after 3hoc101_7Zcoa_m
after C31H48N7O18P3S
after BiGG ID not found in BiGG
..............................................
..............................................
..............................................
before 3hoc101_7Zcoa_x
before C31H48N7O18P3S
before 
..............................................
after 3hoc101_7Zcoa_x
after C31H48N7O18P3S
after BiGG ID not found in BiGG
..............................................
..............................................
..............................................
before 3hoc121_6Ecoa_m
before C33H52N7O18P3S
before 
..............................................
after 3hoc121_6Ecoa_m
after C33H52N7O18P3S
after BiGG ID not found in BiGG
..............................................
..............................................
..............................................
before 3hoc122_6Z_9Zcoa_m
before C33H50N7O18P3S
befo

/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4175/3971818088.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unkown_mets['Name'].iloc[Met_Counter] = Descriptive_Names[Met_Counter]
/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4175/3971818088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unkown_mets['Formula'].iloc[Met_Counter] = Formulae[Met_Counter]


In [68]:
metabolites.update(unkown_mets)

# Manual Curation
for bigg_id in metabolites['BiGG ID']:
    # xtra = Xanthurenic acid; C10H6NO4
    # http://bigg.ucsd.edu/models/iCHOv1/reactions/r0647
    if 'xtra' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'Xanthurenic acid'
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Formula'] = 'C10H6NO4'
    # chedxch = Bilirubin-monoglucuronoside; C39H42N4O122-
    # Reactions name = 'ATP-binding Cassette (ABC) TCDB:3.A.1.208.2' --> https://metabolicatlas.org/identifier/TCDB/3.A.1.208.2
    elif 'chedxch' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'Bilirubin-monoglucuronoside'
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Formula'] = 'C39H42N4O122-'
    # chatGTP
    elif '3hoc246_6Z_9Z_12Z_15Z_18Z_21Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a 24-carbon fatty acid with six double bonds, with the location of the double bonds specified by the numbers and Zs'
    # chatGTP
    elif 'c247_2Z_6Z_9Z_12Z_15Z_18Z_21Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a modified version of the same 24-carbon fatty acid, with a hydroxyl group added at the third carbon position'
    # chatGTP
    elif '3hoc143_5Z_8Z_11Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a 14-carbon fatty acid with three double bonds, with the location of the double bonds specified by the numbers and Zs.'
    # chatGTP
    elif '3oc143_5Z_8Z_11Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a modified version of the same 14-carbon fatty acid, with the hydroxyl group removed and one of the double bonds converted to a keto group'
    # chatGTP
    elif 'acgalgalacglcgalgluside' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'Complex glycosphingolipid that contains multiple sugar residues'

    # 12e8hdx WTF?
    # hdxur Dead End

metabolites.to_excel('../Data/Metabolites/metabolites_final.xlsx')

/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4175/2563423044.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unkown_mets['Name'].iloc[Met_Counter] = Descriptive_Names[Met_Counter]
/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4175/2563423044.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unkown_mets['Formula'].iloc[Met_Counter] = Formulae[Met_Counter]


## C. Addition of metabolites from Recon3D
The idea here is to add the metabolites from the reactions added from Recon 3D, at the same time that we do not over write data from our own Metabolites dataset.

In [21]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction + Recon3D')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
rxns_sheet = cho_recon.worksheet('Rxns')

In [22]:
# We can extract the data using the get_all_records method and create a pd DataFrame
df = pd.DataFrame(rxns_sheet.get_all_records())
df

,Curated,Reaction,Reaction Name,Reaction Formula,Subsystem,GPR_hef,GPR_fou,GPR_yeo,GPR_Recon3D,GPR_final,Conf. Score,Curation Notes,References
0,PD,10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal",10fthf5glu_c --> 10fthf5glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,
1,PD,10FTHF5GLUtm,"5-glutamyl-10FTHF transport, mitochondrial",10fthf5glu_m --> 10fthf5glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,
2,PD,10FTHF6GLUtl,"6-glutamyl-10FTHF transport, lysosomal",10fthf6glu_c --> 10fthf6glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,
3,PD,10FTHF6GLUtm,"6-glutamyl-10FTHF transport, mitochondrial",10fthf6glu_m --> 10fthf6glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,
4,PD,10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal",10fthf7glu_c --> 10fthf7glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11907,,r2436,Mitochondrial Carrier (Mc) Tcdb:2.A.29.8.3,crn_m + odecrn_c <=> crn_c + odecrn_m,"Transport, mitochondrial",,,,,100765000,,,
11908,,r2437,Mitochondrial Carrier (Mc) Tcdb:2.A.29.8.3,crn_m + pcrn_c <=> crn_c + pcrn_m,"Transport, mitochondrial",,,,,100765000,,,
11909,,r2438,Mitochondrial Carrier (Mc) Tcdb:2.A.29.8.3,c4crn_c + crn_m <=> c4crn_m + crn_c,"Transport, mitochondrial",,,,,100765000,,,
11910,,r2439,"Zinc (Zn2+)-Iron (Fe2+) Permease (Zip), Tcdb:2...",hco3_c + so4_e --> hco3_e + so4_c,"Transport, extracellular",,,,,103159861,,,


In [23]:
# Create a cobra model to identify the metabolites involved in our reconstruction
model = cobra.Model("iCHOxxxx")
lr = []

for _, row in df.iterrows():
    r = cobra.Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

Name,iCHOxxxx
Memory address,168b77b80
Number of metabolites,0
Number of reactions,11912
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [24]:
# With the built in function "build_reaction_from_string" we can identify the metabolites
for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(df['Reaction Formula'][i])
    print(r)

  0%|          | 0/11912 [00:00<?, ?it/s]

unknown metabolite '10fthf5glu_c' created
unknown metabolite '10fthf5glu_l' created
10FTHF5GLUtl: 10fthf5glu_c --> 10fthf5glu_l
unknown metabolite '10fthf5glu_m' created
10FTHF5GLUtm: 10fthf5glu_m --> 10fthf5glu_c
unknown metabolite '10fthf6glu_c' created
unknown metabolite '10fthf6glu_l' created
10FTHF6GLUtl: 10fthf6glu_c --> 10fthf6glu_l
unknown metabolite '10fthf6glu_m' created
10FTHF6GLUtm: 10fthf6glu_m --> 10fthf6glu_c
unknown metabolite '10fthf7glu_c' created
unknown metabolite '10fthf7glu_l' created
10FTHF7GLUtl: 10fthf7glu_c --> 10fthf7glu_l
unknown metabolite '10fthf7glu_m' created
10FTHF7GLUtm: 10fthf7glu_m --> 10fthf7glu_c
unknown metabolite '10fthf_c' created
unknown metabolite '10fthf_l' created
10FTHFtl: 10fthf_c <=> 10fthf_l
unknown metabolite '10fthf_m' created
10FTHFtm: 10fthf_c <=> 10fthf_m
unknown metabolite '11docrtsl_c' created
unknown metabolite '11docrtsl_m' created
11DOCRTSLtm: 11docrtsl_c <=> 11docrtsl_m
unknown metabolite '11docrtsl_r' created
11DOCRTSLtr: 11d

unknown metabolite 'L2aadp6sa_m' created
AASAD3m: L2aadp6sa_m + h2o_m + nad_m --> L2aadp_m + 2.0 h_m + nadh_m
AATAi: L2aadp_c + akg_c <=> 2oxoadp_c + glu_L_c
unknown metabolite 'chedxch_c' created
unknown metabolite 'chedxch_e' created
ABC_TCDBd: atp_c + chedxch_c + h2o_c --> adp_c + chedxch_e + h_c + pi_c
unknown metabolite 'fucgalacglcgalgluside_cho_g' created
unknown metabolite 'udpacgal_g' created
unknown metabolite 'acgalfucgalacglcgalgluside_cho_g' created
ABO1g: fucgalacglcgalgluside_cho_g + udpacgal_g --> acgalfucgalacglcgalgluside_cho_g + h_g + udp_g
unknown metabolite 'galfucgalacglcgalgluside_cho_g' created
ABO2g: fucgalacglcgalgluside_cho_g + udpgal_g --> galfucgalacglcgalgluside_cho_g + h_g + udp_g
unknown metabolite 'fuc12gal14acglcgalgluside_cho_g' created
unknown metabolite 'galfuc12gal14acglcgalgluside_cho_g' created
ABO3g: fuc12gal14acglcgalgluside_cho_g + udpgal_g --> galfuc12gal14acglcgalgluside_cho_g + h_g + udp_g
unknown metabolite 'acgalfuc12gal14acglcgalgluside_

unknown metabolite 'lnlncg_x' created
ACOATE1831p: h2o_x + lnlncgcoa_x --> coa_x + h_x + lnlncg_x
unknown metabolite 'lnlncacoa_c' created
unknown metabolite 'lnlnca_c' created
ACOATE1832: h2o_c + lnlncacoa_c --> coa_c + h_c + lnlnca_c
unknown metabolite 'lnlnca_m' created
ACOATE1832m: h2o_m + lnlncacoa_m --> coa_m + h_m + lnlnca_m
unknown metabolite 'lnlnca_x' created
ACOATE1832p: h2o_x + lnlncacoa_x --> coa_x + h_x + lnlnca_x
unknown metabolite 'strdnccoa_c' created
unknown metabolite 'strdnc_c' created
ACOATE184: h2o_c + strdnccoa_c --> coa_c + h_c + strdnc_c
unknown metabolite 'strdnc_m' created
ACOATE184m: h2o_m + strdnccoa_m --> coa_m + h_m + strdnc_m
unknown metabolite 'strdnc_x' created
ACOATE184p: h2o_x + strdnccoa_x --> coa_x + h_x + strdnc_x
unknown metabolite 'eicostricoa_c' created
unknown metabolite 'eicostri_c' created
ACOATE203: eicostricoa_c + h2o_c --> coa_c + eicostri_c + h_c
unknown metabolite 'dlnlcgcoa_c' created
unknown metabolite 'dlnlcg_c' created
ACOATE2031: d

unknown metabolite 'akgp_cho_c' created
AGLPR: akdhap_cho_c + h_c + nadph_c --> akgp_cho_c + nadp_c
unknown metabolite 'Rtotal2_c' created
AGLPT: Rtotal2_c + akgp_cho_c + h_c --> ak2gp_cho_c + h2o_c
unknown metabolite 'thbpt_c' created
unknown metabolite 'dhbpt_c' created
unknown metabolite 'oak2g_cho_c' created
AGMO: ak2g_cho_c + o2_c + thbpt_c --> dhbpt_c + h2o_c + oak2g_cho_c
unknown metabolite 'agm_m' created
unknown metabolite 'ptrc_m' created
unknown metabolite 'urea_m' created
AGMTm: agm_m + h2o_m --> ptrc_m + urea_m
unknown metabolite 'Rtotal2coa_c' created
unknown metabolite 'alpa_cho_c' created
unknown metabolite 'pa_cho_c' created
AGPAT1: Rtotal2coa_c + alpa_cho_c --> coa_c + pa_cho_c
unknown metabolite 'Rtotal2coa_m' created
unknown metabolite 'alpa_cho_m' created
AGPAT1m: Rtotal2coa_m + alpa_cho_m --> coa_m + pa_cho_m
unknown metabolite 'acg5p_m' created
unknown metabolite 'acg5sa_m' created
AGPRim: acg5p_m + h_m + nadph_m --> acg5sa_m + nadp_m + pi_m
unknown metabolite 'a

...

TCA3Sabc: atp_c + h2o_c + tca3s_c --> adp_c + h_c + pi_c + tca3s_e
TCA3St: hco3_c + tca3s_e <=> hco3_e + tca3s_c
TCASULT: paps_c + tchola_c --> h_c + pap_c + tca3s_c
unknown metabolite 'tcdca3s_c' created
unknown metabolite 'tcdca3s_e' created
TCDCA3Sabc: atp_c + h2o_c + tcdca3s_c --> adp_c + h_c + pi_c + tcdca3s_e
TCDCA3St: hco3_c + tcdca3s_e <=> hco3_e + tcdca3s_c
unknown metabolite 'thyochol_c' created
TCDCA6AH: nadph_c + o2_c + tdchola_c --> h2o_c + nadp_c + thyochol_c
TCDCASULT: paps_c + tdchola_c --> h_c + pap_c + tcdca3s_c
TCHOLABCtc: atp_c + h2o_c + tchola_e --> adp_c + h_c + pi_c + tchola_c
unknown metabolite 'tdca3s_c' created
unknown metabolite 'tdca3s_e' created
TDCA3Sabc: atp_c + h2o_c + tdca3s_c --> adp_c + h_c + pi_c + tdca3s_e
TDCA3St: hco3_c + tdca3s_e <=> hco3_e + tdca3s_c
TDCASULT: paps_c + tdechola_c --> h_c + pap_c + tdca3s_c
unknown metabolite 'tetdecaeth_e' created
TETDECAETH: h2o_e + pe12_hs_e --> pa_hs_e + tetdecaeth_e
unknown metabolite 'thexdd_m' created
THEX

unknown metabolite 'tyrargser_c' created
TYRARGSERt: h_e + tyrargser_e <=> h_c + tyrargser_c
unknown metabolite 'melanin_c' created
TYRASE: 56dihindlcrbxlt_c --> melanin_c
unknown metabolite 'tyrasparg_e' created
unknown metabolite 'tyrasparg_c' created
TYRASPARGt: h_e + tyrasparg_e <=> h_c + tyrasparg_c
TYRB0AT3tc: cl_e + na1_e + tyr_L_e --> cl_c + na1_c + tyr_L_c
unknown metabolite 'tyrcysgly_e' created
unknown metabolite 'tyrcysgly_c' created
TYRCYSGLYt: h_e + tyrcysgly_e <=> h_c + tyrcysgly_c
unknown metabolite 'tyrcysthr_e' created
unknown metabolite 'tyrcysthr_c' created
TYRCYSTHRt: h_e + tyrcysthr_e <=> h_c + tyrcysthr_c
unknown metabolite 'ind56qn_c' created
TYRDHINDOX: 2.0 CE4888_c + o2_c --> 2.0 h2o_c + 2.0 ind56qn_c
TYRDOPOX: 2.0 dopa_c + o2_c --> 2.0 CE5276_c + 2.0 h2o_c
unknown metabolite 'tyrglu_e' created
unknown metabolite 'tyrglu_c' created
TYRGLUt: h_e + tyrglu_e <=> h_c + tyrglu_c
unknown metabolite 'tyrleuarg_e' created
unknown metabolite 'tyrleuarg_c' created
TYRLE

r0666: atp_c + fpram_c <=> adp_c + air_c + 2.0 h_c + pi_c
unknown metabolite 'HC01223_m' created
r0669: 2mp2coa_m + h2o_m --> HC01223_m
unknown metabolite 'HC01587_c' created
r0678: acACP_c + h_c + malACP_c --> ACP_c + HC01587_c + co2_c
unknown metabolite 'HC01321_c' created
unknown metabolite 'HC01255_c' created
r0681: HC01321_c --> HC01255_c + h2o_c
unknown metabolite 'HC01588_c' created
r0682: HC01588_c + nadp_c --> HC01255_c + h_c + nadph_c
r0683: ppcoa_m + q10_m --> prpncoa_m + q10h2_m
r0688: h2o_c + nad_c + xol7ah2al_c <=> dhcholestanate_c + 2.0 h_c + nadh_c
r0691: HC01321_c + nadp_c --> HC01587_c + h_c + nadph_c
unknown metabolite 'HC01322_c' created
unknown metabolite 'HC01596_c' created
r0692: HC01322_c + nadp_c --> HC01596_c + h_c + nadph_c
unknown metabolite 'HC01597_c' created
r0693: HC01322_c --> HC01597_c + h2o_c
unknown metabolite 'HC01323_c' created
unknown metabolite 'HC01593_c' created
r0694: HC01323_c + nadp_c --> HC01593_c + h_c + nadph_c
unknown metabolite 'HC01594

unknown metabolite 'HC01942_c' created
r1103: 37.0 ala_L_c + 42.0 arg_L_c + 40.0 asn_L_c + 36.0 asp_L_c + 3240.0 atp_c + 48.0 cys_L_c + 31.0 gln_L_c + 56.0 glu_L_c + 62.0 gly_c + 3240.0 h2o_c + 24.0 his_L_c + 22.0 ile_L_c + 48.0 leu_L_c + 49.0 lys_L_c + 11.0 met_L_c + 21.0 phe_L_c + 69.0 pro_L_c + 56.0 ser_L_c + 61.0 thr_L_c + 19.0 trp_L_c + 30.0 tyr_L_c + 48.0 val_L_c --> HC01942_c + 2430.0 adp_c + 810.0 amp_c + 2430.0 pi_c + 810.0 ppi_c
unknown metabolite 'HC01943_c' created
r1104: 42.0 ala_L_c + 44.0 arg_L_c + 25.0 asn_L_c + 35.0 asp_L_c + 2488.0 atp_c + 26.0 cys_L_c + 26.0 gln_L_c + 51.0 glu_L_c + 49.0 gly_c + 2488.0 h2o_c + 13.0 his_L_c + 22.0 ile_L_c + 51.0 leu_L_c + 29.0 lys_L_c + 9.0 met_L_c + 21.0 phe_L_c + 33.0 pro_L_c + 38.0 ser_L_c + 36.0 thr_L_c + 14.0 trp_L_c + 21.0 tyr_L_c + 37.0 val_L_c --> HC01943_c + 1866.0 adp_c + 622.0 amp_c + 1866.0 pi_c + 622.0 ppi_c
unknown metabolite 'HC01944_c' created
r1105: 61.0 ala_L_c + 27.0 arg_L_c + 34.0 asn_L_c + 45.0 asp_L_c + 2792.0 at

Name,iCHOxxxx
Memory address,168b77b80
Number of metabolites,8666
Number of reactions,11912
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [26]:
model

Name,iCHOxxxx
Memory address,168b77b80
Number of metabolites,8666
Number of reactions,11912
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [27]:
# We first create a list of the metabolites and then a pandas df with it
metabolites_cho_recon3d_list = []
for met in model.metabolites:
    metabolites_cho_recon3d_list.append(met.id)
    
metabolites_cho_recon3d = pd.DataFrame(metabolites_cho_recon3d_list, columns =['BiGG ID'])
metabolites_cho_recon3d

,BiGG ID
0,10fthf5glu_c
1,10fthf5glu_l
2,10fthf5glu_m
3,10fthf6glu_c
4,10fthf6glu_l
...,...
8661,HC02098_c
8662,HC02099_c
8663,HC01988_c
8664,HC10856_m


In [35]:
metabolites_cho_recon3d['BiGG ID']

array(['10fthf5glu_c', '10fthf5glu_l', '10fthf5glu_m', ..., 'HC01988_c',
       'HC10856_m', 'HC10857_m'], dtype=object)

In [32]:
# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
metabolites_sheet = cho_recon.worksheet('Metabolites')
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())

,Curated,BiGG ID,Name,Formula,Compartment,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
0,,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c - cytosol,,,,,,,,,,,,,,,
1,,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l - lysosome,,,,,,,,,,,,,,,
2,,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m - mitochondria,,,,,,,,,,,,,,,
3,,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c - cytosol,,,,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,,,,,,,,,,,
4,,10fthf6glu_l,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,l - lysosome,,,,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5426,,HC02216_c,Prostaglandin-f2beta,C20H33O5,c - cytosol,,28922,439702,InChI=1S/C20H34O5/c1-2-3-6-9-15(21)12-13-17-16...,,,CCCCC[C@H](O)\C=C\[C@H]1[C@H](O)C[C@@H](O)[C@@...,InChI=1S/C20H34O5/c1-2-3-6-9-15(21)12-13-17-16...,,,Neutral,5280506,,https://pubchem.ncbi.nlm.nih.gov/compound/5280506,https://www.ebi.ac.uk/chebi/searchId.do;jsessi...
5427,,HC01361_e,Dihydroneopterin,C9H13N5O4,e - extracellular space,C04874,17001,,InChI=1S/C9H13N5O4/c10-9-13-7-5(8(18)14-9)12-3...,,,Nc1nc2c(c(=O)[nH]1)N=C(C(O)C(O)CO)CN2,,,,,,,,
5428,,HC02217_c,Prostaglandin-g2,C20H31O6,c - cytosol,,27647,,InChI=1S/C20H32O6/c1-2-3-6-9-15(24-23)12-13-17...,,C05956,CCCCC[C@H](OO)\C=C\[C@H]1[C@H]2C[C@H](OO2)[C@@...,InChI=1S/C20H32O6/c1-2-3-6-9-15(24-23)12-13-17...,PGX,,Neutral,5280883,http://ligand-expo.rcsb.org/pyapps/ldHandler.p...,https://pubchem.ncbi.nlm.nih.gov/compound/5280883,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...
5429,PD,fdxo_2_2_c,Oxidized ferredoxin,Fe2S2X,c - cytosol,C00139,17908,,,,,[*][*][Fe+3]1([*][*])[S-2][Fe+3]([*][*])([*][*...,,,,,,,,


In [36]:
subset = metabolites_cho_recon3d[~metabolites_cho_recon3d['BiGG ID'].isin(metabolites['BiGG ID'])].dropna()
subset = subset.reset_index(drop=True)

In [38]:
subset

,BiGG ID
0,11docrtsl_e
1,11docrtstrn_e
2,12dhchol_c
3,12dhchol_e
4,tacr_r
...,...
3230,HC02098_c
3231,HC02099_c
3232,HC01988_c
3233,HC10856_m


In [ ]:
# Transform the final Recon3D Metabolites dataset into a dictionary to map it into our dataset
final_met_dict = df_to_dict(models_metabolites, 'BiGG ID')

In [61]:
final_met_dict

{'10fthf5glu_c': {'Name': '10-formyltetrahydrofolate-[Glu](5)',
  'Formula': 'C40H45N11O19',
  'Compartment': 'c',
  'KEGG': nan,
  'CHEBI': nan,
  'PubChem': nan,
  'Inchi': nan,
  'Hepatonet': nan,
  'EHMNID': nan,
  'SMILES': nan,
  'INCHI2': nan,
  'CC_ID': nan,
  'Stereoisomer Information of Metabolite Identified': nan,
  'Charge of the Metabolite Identified': nan,
  'CID_ID': nan,
  'PDB (ligand-expo) Experimental Coordinates  File Url': nan,
  'Pub Chem Url': nan,
  'ChEBI Url': nan},
 '10fthf5glu_e': {'Name': '10-formyltetrahydrofolate-[Glu](5)',
  'Formula': 'C40H45N11O19',
  'Compartment': 'e',
  'KEGG': nan,
  'CHEBI': nan,
  'PubChem': nan,
  'Inchi': nan,
  'Hepatonet': nan,
  'EHMNID': nan,
  'SMILES': nan,
  'INCHI2': nan,
  'CC_ID': nan,
  'Stereoisomer Information of Metabolite Identified': nan,
  'Charge of the Metabolite Identified': nan,
  'CID_ID': nan,
  'PDB (ligand-expo) Experimental Coordinates  File Url': nan,
  'Pub Chem Url': nan,
  'ChEBI Url': nan},
 '10ft

In [62]:
subset[['Name', 'Formula', 'Compartment', 'KEGG','CHEBI', 'PubChem','Inchi', 'Hepatonet', 'EHMNID', 'SMILES',
             'INCHI2','CC_ID','Stereoisomer Information of Metabolite Identified', 'Charge of the Metabolite Identified',
    'CID_ID','PDB (ligand-expo) Experimental Coordinates  File Url', 'Pub Chem Url',
    'ChEBI Url']] = subset['BiGG ID'].apply(lambda x: pd.Series(final_met_dict.get(x, None), dtype=object))

In [63]:
subset

,BiGG ID,Name,Formula,Compartment,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
0,11docrtsl_e,11docrtsl c,C21H30O4,e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11docrtstrn_e,11-Deoxycorticosterone,C21H30O3,e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12dhchol_c,12-Dehydrocholic acid; 12-Oxodeoxycholic acid;...,C24H37O5,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12dhchol_e,12-Dehydrocholic acid; 12-Oxodeoxycholic acid;...,C24H37O5,e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tacr_r,Tacrolimus,C44H69NO12,r,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3230,HC02098_c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3231,HC02099_c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3232,HC01988_c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3233,HC10856_m,"Trans,Cis-Octadeca-2,9-Dienoyl Coenzyme A",C39H62N7O17P3S,m,NaN,NaN,NaN,NaN,NaN,NaN,CCCCCCCC\C=C/CCCCC\C=C\C(=O)SCCNC(=O)CCNC(=O)[...,InChI=1S/C39H66N7O17P3S/c1-4-5-6-7-8-9-10-11-1...,NaN,NaN,Neutral,86289273.0,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/8628...,https://www.ebi.ac.uk/chebi/searchId.do;jsessi...


In [65]:
subset.to_excel('../Data/Metabolites/metabolites_recon3d_toadd.xlsx')